In [13]:
import pandas as pd
total= pd.read_csv("ffinal_total4.csv")

In [14]:
total.columns

Index(['Unnamed: 0', '기준년분기코드', '행정동코드', '월평균소득금액', '음식지출총금액', '유사업종점포수',
       '개업점포수', '폐업점포수', '당월매출금액', '총직장인구수', '총상주인구수', '총유동인구수', '환산전체',
       '환산1층', '환산그외'],
      dtype='object')

In [15]:
# required libraries
import numpy as np
np.random.seed(42)
import os
import urllib.request
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import pyplot
import seaborn as sns
import joblib

# machine learning libraries
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import r2_score
from scipy.stats import randint
# preprocessing 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
# date
from datetime import date
import datetime

In [16]:
#train,test
y = total[['당월매출금액']]
X = total[[ '월평균소득금액', '음식지출총금액', '유사업종점포수', '총직장인구수', '총상주인구수', '총유동인구수', '환산전체', '환산1층', '환산그외']]

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)
print("X_train: ", X_train.shape), print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape), print("y_test: ", y_test.shape)

X_train:  (6364, 9)
y_train:  (6364, 1)
X_test:  (1591, 9)
y_test:  (1591, 1)


(None, None)

In [18]:
#스케일링
scaler = StandardScaler()
scaler.fit(X_train)
X_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
print(X_scaled[:1])
print("----------------")
print(X_test_scaled[:1])

[[0.21059596 7.29847769 8.08630626 6.93954026 1.42202878 5.51466223
  0.82326055 0.59161137 0.9236374 ]]
----------------
[[-0.72439391 -0.38803414 -0.43159461 -0.44797811 -1.14614715  0.05088147
  -0.33453936  0.18344846 -0.86911198]]


In [19]:
#RMSE확인
lin_reg = LinearRegression()
lin_reg.fit(X_scaled, y_train)
some_data = X_scaled[:5]
some_labels = y_train[:5]
print("Predictions:", lin_reg.predict(some_data))
print("Labels:", list(some_labels))

Predictions: [[8.04205556e+10]
 [4.14467307e+09]
 [6.64074797e+09]
 [4.42255765e+09]
 [2.32573999e+10]]
Labels: ['당월매출금액']


In [20]:
from sklearn.ensemble import RandomForestRegressor 

In [21]:

forest_reg = RandomForestRegressor(n_estimators=200, random_state=42)
forest_reg.fit(X_scaled, y_train)

forest_predictions = forest_reg.predict(X_scaled)
forest_mse = mean_squared_error(y_train, forest_predictions)
forest_rmse = np.sqrt(forest_mse)
#
#test
forest_test_predictions = forest_reg.predict(X_test_scaled)
forest_test_mse = mean_squared_error(y_test, forest_test_predictions)
forest_test_rmse = np.sqrt(forest_test_mse)
#
forest_scores = cross_val_score(forest_reg, X_scaled, y_train,
                                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
#
print("<<<Random Forest>>>")
print(forest_rmse_scores)
print("forest_rmse:", forest_rmse)
print("-----------------------------------------------")
print("R2 score on training set : ", r2_score(y_train, forest_predictions))
print("R2 score on test set : ", r2_score(y_test, forest_test_predictions))

c:\Users\jeong\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\jeong\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\jeong\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\jeong\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:1473: DataConvers

<<<Random Forest>>>
[1.73582546e+09 1.20490667e+09 1.51910087e+09 1.77114654e+09
 1.39241835e+09 1.25835348e+09 1.56471826e+09 1.64928779e+09
 1.40715199e+09 1.54085407e+09]
forest_rmse: 559229972.4899927
-----------------------------------------------
R2 score on training set :  0.9966560454507226
R2 score on test set :  0.9791323284307786


In [22]:
#예측값 뽑아내기
reg = RandomForestRegressor(n_estimators=200, random_state = 42).fit(X_scaled, y_train)
#피클파일로 저장

joblib.dump(reg, 'reg.pkl', compress=1)

c:\Users\jeong\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


['reg.pkl']

In [23]:
# Evaluation
loaded_model = joblib.load('./reg.pkl')
#
score = loaded_model.score(X_test_scaled, y_test)
print('test score: {score:.3f}'.format(score=score))

test score: 0.979


In [24]:
df = pd.DataFrame(X_test, columns = ['월평균소득금액', '음식지출총금액', '유사업종점포수', '총직장인구수', '총상주인구수', '총유동인구수','환산전체', '환산1층', '환산그외'])
#
predictions = reg.predict(X_test_scaled)
df['prediction_rf'] = predictions
#
# 소수점 2자리로 매출액 표시
pd.options.display.float_format = '{:.2f}'.format


In [25]:
df

,월평균소득금액,음식지출총금액,유사업종점포수,총직장인구수,총상주인구수,총유동인구수,환산전체,환산1층,환산그외,prediction_rf
1650,2722774,800191000,93,1421,12359,5800073,100194,148726,51663,3570413618.05
3182,3333112,2043928000,230,6669,16799,7888102,117679,153779,81579,8119454482.27
251,2747467,2109396000,194,2498,23709,7236595,106667,123159,90175,7327410439.35
453,3416366,1282355000,137,3314,30158,4433577,99016,133751,64281,5771913977.69
6821,3682946,3787935000,173,13034,7126,2641988,182180,213394,150966,8671870319.20
...,...,...,...,...,...,...,...,...,...,...
3856,4622839,441903000,84,3997,15900,2116351,119337,145136,93539,1732213204.83
5138,3225725,3601761000,222,10344,15664,6199575,152235,187553,116917,11816249843.88
1593,3381883,1306234000,156,22158,40147,11528475,79773,97690,61856,6372912031.99
2184,5384412,267486000,21,18155,25265,1871358,146547,204418,88675,159465436.58


In [29]:
df.columns

Index(['월평균소득금액', '음식지출총금액', '유사업종점포수', '총직장인구수', '총상주인구수', '총유동인구수', '환산전체',
       '환산1층', '환산그외', 'prediction_rf'],
      dtype='object')

In [31]:
total_predict = df.merge(total, on = ['월평균소득금액', '음식지출총금액', '유사업종점포수', '총직장인구수', '총상주인구수', '총유동인구수', '환산전체',
       '환산1층', '환산그외'],how='inner')

In [32]:
total_predict

,월평균소득금액,음식지출총금액,유사업종점포수,총직장인구수,총상주인구수,총유동인구수,환산전체,환산1층,환산그외,prediction_rf,Unnamed: 0,기준년분기코드,행정동코드,개업점포수,폐업점포수,당월매출금액
0,2722774,800191000,93,1421,12359,5800073,100194,148726,51663,3570413618.05,1650,20201,11230720,1,6,2302043834
1,3333112,2043928000,230,6669,16799,7888102,117679,153779,81579,8119454482.27,3182,20211,11110650,5,3,6392837438
2,2747467,2109396000,194,2498,23709,7236595,106667,123159,90175,7327410439.35,251,20191,11620735,15,9,3719447268
3,3416366,1282355000,137,3314,30158,4433577,99016,133751,64281,5771913977.69,453,20192,11380520,11,8,6114086699
4,3682946,3787935000,173,13034,7126,2641988,182180,213394,150966,8671870319.20,6821,20232,11170650,4,1,9117761871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1586,4622839,441903000,84,3997,15900,2116351,119337,145136,93539,1732213204.83,3856,20212,11410690,0,0,1946475303
1587,3225725,3601761000,222,10344,15664,6199575,152235,187553,116917,11816249843.88,5138,20221,11620585,3,7,10213339701
1588,3381883,1306234000,156,22158,40147,11528475,79773,97690,61856,6372912031.99,1593,20201,11290705,5,3,6129526514
1589,5384412,267486000,21,18155,25265,1871358,146547,204418,88675,159465436.58,2184,20202,11470670,0,0,146382795


In [33]:
median = total_predict.groupby('행정동코드')[['당월매출금액']].median().reset_index().rename(columns={'당월매출금액' : 'median'})

In [34]:
ffinal = median.merge(total_predict, on = ['행정동코드'],how = 'inner')

In [35]:
ffinal['success'] = ffinal.apply(lambda row: 0 if row['median'] > row['prediction_rf'] else 1, axis=1)

In [36]:
ffinal

,행정동코드,median,월평균소득금액,음식지출총금액,유사업종점포수,총직장인구수,총상주인구수,총유동인구수,환산전체,환산1층,환산그외,prediction_rf,Unnamed: 0,기준년분기코드,개업점포수,폐업점포수,당월매출금액,success
0,11110515,2111867516.00,3780222,636006000,83,10628,12817,3612274,130611,165848,95374,2899745185.34,1618,20201,1,7,2061038996,1
1,11110515,2111867516.00,3780222,936908000,84,10628,12434,3759011,125839,163704,87974,3064907447.68,3859,20212,3,6,2756910122,1
2,11110515,2111867516.00,3780222,857237000,89,10628,12434,3595242,131721,183465,79977,3637299885.09,4085,20213,8,3,2381121646,1
3,11110515,2111867516.00,3780222,787974000,84,10628,12817,3706408,131133,160242,102023,2369635743.91,2031,20202,1,0,2102339126,1
4,11110515,2111867516.00,4015674,836137000,83,10997,13030,2552571,110935,144858,77012,3492599801.03,473,20192,2,2,2111867516,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1586,11740700,5213130074.00,3603496,1098479000,119,8575,28372,7944260,66322,88565,44080,5621411660.99,852,20193,7,2,5178034962,1
1587,11740700,5213130074.00,3414406,1122368000,106,4961,26347,7182157,81769,113461,50077,5345342673.48,5200,20222,2,5,6441320449,1
1588,11740700,5213130074.00,3414406,918173000,117,4961,27648,8380409,70079,91457,48702,5353462912.06,1934,20201,4,6,4288305740,1
1589,11740700,5213130074.00,3414406,1191852000,104,4961,26347,7207926,93867,109900,77833,6064968952.56,5924,20223,2,4,6522555125,1


In [38]:
ffinal.to_csv("ffinal_standard_Randomforest.csv",index=False)

In [39]:
# 교차 검증으로 훈련 세트의 평균 제곱 오차 (MSE) 계산
train_cv_scores = cross_val_score(forest_reg , X_train, y_train, cv=5, scoring='neg_mean_squared_error')
train_mse = -np.mean(train_cv_scores)  # 부호 변경하여 양수로 변환
print("Train CV MSE:", train_mse)

# 교차 검증으로 테스트 세트의 평균 제곱 오차 (MSE) 계산
test_cv_scores = cross_val_score(forest_reg , X_test, y_test, cv=5, scoring='neg_mean_squared_error')
test_mse = -np.mean(test_cv_scores)  # 부호 변경하여 양수로 변환
print("Test CV MSE:", test_mse)


# 과적합 여부 확인
if train_mse < test_mse:
    print("과적합 아닙니다")
else:
    print("과적합입니다")

c:\Users\jeong\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\jeong\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\jeong\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\jeong\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:1473: DataConvers

Train CV MSE: 2.597941196400349e+18


c:\Users\jeong\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\jeong\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\jeong\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\jeong\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:1473: DataConvers

Test CV MSE: 1.1944016330676124e+19
과적합 아닙니다
